<a href="https://colab.research.google.com/github/akiroh0128/projectgigs/blob/main/Neural_Network_SMS_Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.6/490.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 45.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


2.14.0-dev20230628


In [2]:
# getting data files from project data of freecodecamp.org
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-06-28 14:55:53--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   877KB/s    in 0.4s    

2023-06-28 14:55:54 (877 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-06-28 14:55:54--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[===========

In [15]:
#Convert the data into pandas dataframes
train_dataset=pd.read_csv(train_file_path,sep='\t',names=['type','message'])
test_dataset=pd.read_csv(test_file_path,sep='\t',names=['type','message'])

#Remove rows with missing data
train_dataset.dropna()
test_dataset.dropna()

batch_size=32

In [16]:
#Label the categorical variables into numercial data
train_dataset["type"] = pd.factorize(train_dataset["type"])[0]
test_dataset["type"] = pd.factorize(test_dataset["type"])[0]

In [17]:
#COnvert the dataframes into tensorflow dataset
train_labels =  train_dataset["type"].values
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_dataset["message"].values, train_labels)
)
test_labels =  test_dataset["type"].values
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_dataset["message"].values, test_labels)
)



In [18]:
#Batching and prefetching the dataset to reshape it
train_dataset = train_dataset.shuffle(100).batch(batch_size).prefetch(tf.data.AUTOTUNE)#shuffling the dataset improves the efficiency of training

test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [19]:
#Create a TextVectorization layer for our model
vec = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_dataset.map(lambda text, label: text))

In [20]:
#Creating the model using keras.Sequential
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


In [21]:
#training the model with our datasets
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_dataset,validation_data=test_dataset, epochs=10,validation_steps=30)

Epoch 1/10
131/131 [==============================] - 60s 279ms/step - loss: 0.2194 - acc: 0.9486 - val_loss: 0.1595 - val_acc: 0.9750
Epoch 2/10
131/131 [==============================] - 23s 174ms/step - loss: 0.1352 - acc: 0.9744 - val_loss: 0.2667 - val_acc: 0.9677
Epoch 3/10
131/131 [==============================] - 24s 186ms/step - loss: 0.1132 - acc: 0.9883 - val_loss: 0.2043 - val_acc: 0.9740
Epoch 4/10
131/131 [==============================] - 24s 187ms/step - loss: 0.1322 - acc: 0.9878 - val_loss: 0.2214 - val_acc: 0.9802
Epoch 5/10
131/131 [==============================] - 22s 173ms/step - loss: 0.1214 - acc: 0.9883 - val_loss: 0.2174 - val_acc: 0.9792
Epoch 6/10
131/131 [==============================] - 24s 184ms/step - loss: 0.1266 - acc: 0.9897 - val_loss: 0.2328 - val_acc: 0.9823
Epoch 7/10
131/131 [==============================] - 22s 173ms/step - loss: 0.0919 - acc: 0.9921 - val_loss: 0.2793 - val_acc: 0.9802
Epoch 8/10
131/131 [==============================] - 2

In [22]:
# function to predict messages based on model
def predict_message(pred_text):
    ps = model.predict([pred_text])
    print(ps)
    p = ps[0][0]
    return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 4s 4s/step
[[-0.95158]]
[-0.95158, 'ham']


In [23]:
# To test our model with some sample text messages
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 36ms/step
[[-0.95158]]
1/1 [==============================] - 0s 36ms/step
[[0.82838434]]
1/1 [==============================] - 0s 40ms/step
[[-1.9831351]]
1/1 [==============================] - 0s 38ms/step
[[3.8660052]]
1/1 [==============================] - 0s 37ms/step
[[3.7640402]]
1/1 [==============================] - 0s 36ms/step
[[-1.4201992]]
1/1 [==============================] - 0s 38ms/step
[[-1.6435294]]
You passed the challenge. Great job!
